## Resultados da Análise Exploratória

Objetivo: Explorar o dataset Breast Cancer Wisconsin e criar modelos baseline simples (Regressão Logística e Árvore de Decisão) para medir recall e entender padrões de dados

Importações das bibliotecas necessárias:

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

### Carregamento do Dataset

In [25]:
# Carrega o dataset Breast Cancer Wisconsin direto do sklearn
data = load_breast_cancer(as_frame=True)

# Cria um DataFrame com os dados para facilitar a analise
df = data.frame
df['target'] = data.target

# Exibe informações gerais
print("Shape (linhas, colunas):", df.shape)
print("\nDistribuição das classes (0 = maligno, 1 = benigno):\n", df['target'].value_counts(normalize=True))

Shape (linhas, colunas): (569, 31)

Distribuição das classes (0 = maligno, 1 = benigno):
 target
1    0.627417
0    0.372583
Name: proportion, dtype: float64


Este dataset possúi 569 amostras com 30 variáveis numéricas que descrevem características de células observadas em imagens microscópicas de biópsias de mama. O objetivo é classificar essas amostras em malignas (câncer) ou benignas (não câncer).

O <b>target</b> indica o dignóstico:
- <b>0</b>: maligno (tumor com câncer)
- <b>1</b>: benigno (tumor não cancerígeno)

```python

### Estrutura e estatísticas descritivas

In [27]:
# Verifica tipos de dados e possíveis valores ausentes
df.info()

# Estatísticas descritivas (média, desvio, mínimo, máximo)
df.describe().T.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

,count,mean,std,min,25%,50%,75%,max
mean radius,569.0,14.127292,3.524049,6.98100,11.70000,13.37000,15.78000,28.11000
mean texture,569.0,19.289649,4.301036,9.71000,16.17000,18.84000,21.80000,39.28000
mean perimeter,569.0,91.969033,24.298981,43.79000,75.17000,86.24000,104.10000,188.50000
mean area,569.0,654.889104,351.914129,143.50000,420.30000,551.10000,782.70000,2501.00000
mean smoothness,569.0,0.096360,0.014064,0.05263,0.08637,0.09587,0.10530,0.16340
mean compactness,569.0,0.104341,0.052813,0.01938,0.06492,0.09263,0.13040,0.34540
mean concavity,569.0,0.088799,0.079720,0.00000,0.02956,0.06154,0.13070,0.42680
mean concave points,569.0,0.048919,0.038803,0.00000,0.02031,0.03350,0.07400,0.20120
mean symmetry,569.0,0.181162,0.027414,0.10600,0.16190,0.17920,0.19570,0.30400
mean fractal dimension,569.0,0.062798,0.007060,0.04996,0.05770,0.06154,0.06612,0.09744


O <b>.iinfo()</b> mostra que todas as colunas são numéricas (<b>float64</b>) e não há valores nulos, excelente qualidade de dados para análise.

O <b>.describe()</b> permite observar escalas diferentes entre colunas, algo importante, pois modelos baseados em distancias ou gradiente são sensíveis à escala (por isso usaremos <b>StandardScaler</b>).

```